In [37]:
import xgboost
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import joblib

In [2]:
from importlib import reload
reload(xgboost)

<module 'xgboost' from '/opt/code/contrib/feast_epam/env/lib/python3.9/site-packages/xgboost/__init__.py'>

In [6]:
df=pd.read_parquet('../features/loan_default_gen/')

In [39]:
df

,LoanID,Age,Income,LoanAmount,CreditScore,MonthsEmployed,NumCreditLines,InterestRate,LoanTerm,DTIRatio,Education,EmploymentType,MaritalStatus,HasMortgage,HasDependents,LoanPurpose,HasCoSigner,Default,user_id,txn_datetime
0,I38PQUQS96,56,85994,50587,520,80,4,15.23,36,0.44,Bachelor's,Full-time,Divorced,Yes,Yes,Other,Yes,0,FOeIRQnsLbpjZvurkMwi,2021-11-10 00:17:55
1,HPSK72WA7R,69,50432,124440,458,15,1,4.81,60,0.68,Master's,Full-time,Married,No,No,Other,Yes,0,kyHoSmrOEWTzDRlpqBVQ,2022-08-27 19:45:08
2,C1OZ6DPJ8Y,46,84208,129188,451,26,3,21.17,24,0.31,Master's,Unemployed,Divorced,Yes,Yes,Auto,No,1,znUAgCQIhmGpWofycevH,2022-09-06 09:18:16
3,V2KKSFM3UN,32,31713,44799,743,0,3,7.07,24,0.23,High School,Full-time,Married,No,No,Business,No,0,PTwkdNvmUazZjcLElrun,2022-10-17 06:17:40
4,EY08JDHTZP,60,20437,9139,633,8,4,6.51,48,0.73,Bachelor's,Unemployed,Divorced,No,Yes,Auto,No,0,rLTKXCBusQyfAMZpzHoS,2021-01-18 18:27:53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255342,8C6S86ESGC,19,37979,210682,541,109,4,14.11,12,0.85,Bachelor's,Full-time,Married,No,No,Other,No,0,muDptexSOYJqzPHiFcLZ,2022-04-11 11:17:09
255343,98R4KDHNND,32,51953,189899,511,14,2,11.55,24,0.21,High School,Part-time,Divorced,No,No,Home,No,1,mewsjcBIpPJFRauTdtNk,2021-09-07 17:19:19
255344,XQK1UUUNGP,56,84820,208294,597,70,3,5.29,60,0.50,High School,Self-employed,Married,Yes,Yes,Auto,Yes,0,SBxlRjhVgndCAtUQuskf,2023-04-27 03:18:58
255345,JAO28CPL4H,42,85109,60575,809,40,1,20.90,48,0.44,High School,Part-time,Single,Yes,Yes,Other,No,0,vyHrhRlCnwjfYkZuBMGm,2023-06-10 10:29:55


In [41]:
df_pivot=df[['LoanID','user_id','txn_datetime','Default','LoanAmount','InterestRate','LoanTerm','DTIRatio']]

In [42]:
import sys
sys.path.append('../')
import os
os.environ['REGISTRY_PATH']='../registry.pb'
os.environ['FEATURE_FOLDER_PATH']='../features'
from pyspark.sql import SparkSession

import pandas as pd
from datetime import datetime

In [43]:
from feature_repo import store

In [44]:
f_store=store.get_store()

/opt/code/contrib/feast_epam/env/lib/python3.9/site-packages/feast/repo_config.py:228: RuntimeWarning: `entity_key_serialization_version` is either not specified in the feature_store.yaml, or is specified to a value <= 1.This serialization version may cause errors when trying to write fields with the `Long` data type into the online store. Specifying `entity_key_serialization_version` to 2 is recommended for new projects. 
  warnings.warn(


In [50]:
df_feature=f_store.get_historical_features(
    entity_df=df_pivot,
    features=[
        'user_info:Age',
        'user_info:Education',
        'user_info:CreditScore',
        'user_info:MonthsEmployed',
        'user_loan_ts:median_previous_income',
        'user_loan_ts:mean_previous_income',
        'user_loan_ts:mean_previous_default',
        ]
).to_df()

/opt/code/contrib/feast_epam/env/lib/python3.9/site-packages/feast/infra/offline_stores/contrib/spark_offline_store/spark_source.py:67: RuntimeWarning: The spark data source API is an experimental feature in alpha development. This API is unstable and it could and most probably will be changed in the future.
  warnings.warn(
/opt/code/contrib/feast_epam/env/lib/python3.9/site-packages/feast/infra/offline_stores/contrib/spark_offline_store/spark.py:137: RuntimeWarning: The spark offline store is an experimental feature in alpha development. Some functionality may still be unstable so functionality can change in the future.
  warnings.warn(


Using txn_datetime as the event timestamp. To specify a column explicitly, please name it event_timestamp.


24/03/22 23:15:03 WARN TaskSetManager: Stage 88 contains a task of very large size (2312 KiB). The maximum recommended task size is 1000 KiB.
24/03/22 23:15:04 WARN TaskSetManager: Stage 89 contains a task of very large size (2312 KiB). The maximum recommended task size is 1000 KiB.


In [56]:
df_feature=df_feature.rename(
    {
        'Default':'default',
        'LoanAmount':'loan_amount',
       'LoanTerm':'loan_term',
        'DTIRatio':'dti_ratio',
        'InterestRate':'interest_rate'
    },
    axis=1
)



In [57]:
target=df_feature[['default']]
feature=df_feature.drop(['LoanID','user_id','txn_datetime','default','Education'],axis=1)

In [59]:
x_train,x_eval,y_train,y_eval=train_test_split(feature,target,train_size=.8)

In [60]:
xgb=xgboost.XGBClassifier(max_depth=100,n_jobs=-1,n_estimators=100)
xgb.fit(x_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=100, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=-1,
              num_parallel_tree=None, random_state=None, ...)

In [61]:
print(classification_report(y_true=y_eval,y_pred=xgb.predict(x_eval)))

              precision    recall  f1-score   support

           0       0.89      0.99      0.93     45132
           1       0.32      0.04      0.07      5938

    accuracy                           0.88     51070
   macro avg       0.60      0.52      0.50     51070
weighted avg       0.82      0.88      0.83     51070



In [62]:
joblib.dump(xgb,'../models/user_xgb_v1.joblib')

['../models/user_xgb_v1.joblib']

In [68]:
xgb.feature_names_in_

array(['loan_amount', 'interest_rate', 'loan_term', 'dti_ratio', 'Age',
       'CreditScore', 'MonthsEmployed', 'median_previous_income',
       'mean_previous_income', 'mean_previous_default'], dtype='<U22')